<a href="https://colab.research.google.com/github/AsB-1/MIAML_Project_3DPointNetClass/blob/main/Ottimizzazione/DEFOttimizzazione_AggiustatoVeramente(forse%3F).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#iperparametri
num_points = 2048
batchsize = 32
feature_transform = True
optimizer_learning_rate = 0.001
optimizer_beta = (0.9, 0.999)
scheduler_step_size = 20
scheduler_gamma = 0.5
drop_rate = 0.3
workers = 4
nepoch = 150
outf = 'cls'

nome_file = "np%d_nb%d_op-lr%.4fb(%.2f,%.4f))_sch-ss%dg%.2f_dr%.2f_ne%d_datasetSbilanciato" % (num_points, batchsize, optimizer_learning_rate, optimizer_beta[0], optimizer_beta[1], scheduler_step_size, scheduler_gamma, drop_rate, nepoch)
print(nome_file)
nome_file = nome_file.replace("0.","")
print(nome_file)

np2048_nb32_op-lr0.0010b(0.90,0.9990))_sch-ss20g0.50_dr0.30_ne150_datasetSbilanciato
np2048_nb32_op-lr0010b(90,9990))_sch-ss20g50_dr30_ne150_datasetSbilanciato


## DATASET

### Import dataset

In [2]:
!wget https://www.roma1.infn.it/~serra/ML/produzione_19042021/20210419_tagli_random_oggetti_esterni.tgz
!wget https://www.roma1.infn.it/~serra/ML/produzione_19042021/20210419_tagli_random_oggetti_interni.tgz

--2023-12-16 12:22:41--  https://www.roma1.infn.it/~serra/ML/produzione_19042021/20210419_tagli_random_oggetti_esterni.tgz
Resolving www.roma1.infn.it (www.roma1.infn.it)... 141.108.26.1, 141.108.26.150
Connecting to www.roma1.infn.it (www.roma1.infn.it)|141.108.26.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18637368 (18M) [application/x-gzip]
Saving to: ‘20210419_tagli_random_oggetti_esterni.tgz’

20210419_tagli_rand 100%[===================>]  17.77M  5.85MB/s    in 3.0s    

2023-12-16 12:22:45 (5.85 MB/s) - ‘20210419_tagli_random_oggetti_esterni.tgz’ saved [18637368/18637368]

--2023-12-16 12:22:45--  https://www.roma1.infn.it/~serra/ML/produzione_19042021/20210419_tagli_random_oggetti_interni.tgz
Resolving www.roma1.infn.it (www.roma1.infn.it)... 141.108.26.1, 141.108.26.150
Connecting to www.roma1.infn.it (www.roma1.infn.it)|141.108.26.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 202494 (198K) [application/x-gzip]
Sav

In [3]:
print("---------Esterni----------------------------------------------------------------------")
!tar xfvz 20210419_tagli_random_oggetti_esterni.tgz
!mkdir esterni
!mv *.stl esterni/
print("---------Interni----------------------------------------------------------------------")
!tar xfvz 20210419_tagli_random_oggetti_interni.tgz
!mkdir interni
!mv *.stl interni/

Output streaming troncato alle ultime 5000 righe.
2021_04_19_18_06_08_Cube_083.stl
2021_04_19_18_06_28_Sphere.stl
2021_04_19_18_06_28_Sphere_002.stl
2021_04_19_18_06_28_Sphere_003.stl
2021_04_19_18_06_28_Sphere_004.stl
2021_04_19_18_06_28_Sphere_005.stl
2021_04_19_18_06_28_Sphere_006.stl
2021_04_19_18_06_28_Sphere_007.stl
2021_04_19_18_06_28_Sphere_008.stl
2021_04_19_18_06_28_Sphere_009.stl
2021_04_19_18_06_28_Sphere_010.stl
2021_04_19_18_06_28_Sphere_011.stl
2021_04_19_18_06_28_Sphere_012.stl
2021_04_19_18_06_28_Sphere_013.stl
2021_04_19_18_06_28_Sphere_014.stl
2021_04_19_18_06_28_Sphere_015.stl
2021_04_19_18_06_28_Sphere_016.stl
2021_04_19_18_06_28_Sphere_017.stl
2021_04_19_18_06_28_Sphere_018.stl
2021_04_19_18_06_28_Sphere_019.stl
2021_04_19_18_06_28_Sphere_020.stl
2021_04_19_18_06_28_Sphere_021.stl
2021_04_19_18_06_28_Sphere_022.stl
2021_04_19_18_06_28_Sphere_023.stl
2021_04_19_18_06_28_Sphere_024.stl
2021_04_19_18_06_28_Sphere_025.stl
2021_04_19_18_06_28_Sphere_026.stl
2021_04_19_

### Coversione trimesh-pointcloud

In [4]:
!pip install trimesh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 688.5/688.5 kB 2.5 MB/s eta 0:00:00


In [ ]:
import os
import glob
import trimesh
import numpy as np
# esempio lettura file e costruzione di array numpy che contengono i punti della pointcloud
pointcloud_interni = np.empty(shape=(0,3,num_points))
pointcloud_esterni = np.empty(shape=(0,3,num_points))


path='interni'
for x in os.listdir(path):
  if x.endswith(".stl"):
    pc = trimesh.load(path+'/'+x).sample(num_points) #converte una figura in pointcloud
    pc = np.transpose(pc)
    pc = np.expand_dims(pc, axis=0) #aggiunge una dimensione(x poterlo concatenare)
    pointcloud_interni = np.concatenate((pointcloud_interni, pc), axis=0) #matrice tridim con tutte le figure(il primo asse indica il label della figura)


label_interni = np.zeros(shape=(pointcloud_interni.shape[0])) #array con zeri della dimensione di quante figure interne abbiamo

path='esterni'
for x in os.listdir(path):
  if x.endswith(".stl"):
    pc = trimesh.load(path+'/'+x).sample(num_points)
    pc = np.transpose(pc)
    pc = np.expand_dims(pc, axis=0)
    pointcloud_esterni = np.concatenate((pointcloud_esterni, pc), axis=0)

label_esterni = np.ones(shape=(pointcloud_esterni.shape[0]))


### Creazione dataset pointcloud

In [ ]:
#dataset con tutti i dati
pointcloud = np.concatenate((pointcloud_interni,pointcloud_esterni), axis=0)
labels = np.concatenate((label_interni,label_esterni), axis=0) # 0 pti interni, 1 pti esterni
#dataset bilanciato
#pointcloud = np.concatenate((pointcloud_interni,pointcloud_esterni[:len(pointcloud_interni),:,:]), axis=0)
#labels = np.concatenate((label_interni,label_esterni[:len(pointcloud_interni)]), axis=0) # 0 pti interni, 1 pti esterni

#!fare lo shuffle, qui sono in ordine interni-esterni
print(pointcloud_interni.shape)
print(label_interni.shape)
print(pointcloud_esterni.shape)
print(label_esterni.shape)
print(pointcloud_esterni[:len(pointcloud_interni),:,:].shape)
print(label_esterni[:len(pointcloud_interni)].shape)
#lo sbilanciamento dei campioni di punti interni ed esterni: non c'è bisogno di rendere i campioni delle label bilanciati perchè comunque ha occasione di allenarsi anche sulle facce lisce presenti nei campioni con label esterne
#c'è anche più varietà tra i tipi di dati esterni: quelli interni sono tutti lisci nello stesso modo mentre quelli esterni posso essere di tipi diversi
#!!!TESTARE
print(pointcloud.shape)
print(labels.shape)

In [ ]:
# salvataggio array numpy pointcloud e labels in un file numpy compresso da salvarsi
# e scaricarsi (per non dover ripetere ogni volta la conversione) sul prorio google drive o sul proprio pc (sono ~170MB)
np.savez('dati_pointcloud', pointcloud=pointcloud, labels=labels)

### Normalizzazione

#### Verifica Normalizzione

In [ ]:
#verifica della normalizzazione e centramento
# salviamo in array le medie per ciascun asse di ogni figura
import matplotlib.pyplot as plt

count = 0
x_mean = []
y_mean = []
z_mean = []

for i in range(pointcloud.shape[0]):
  fig = pointcloud[i, :, :]
  #print("fig.shape = ", fig.shape)
  max = np.max(fig)
  min = np.min(fig)

  #verifica centramento
  x_mean.append(np.mean(fig[0,:]))
  y_mean.append(np.mean(fig[1,:]))
  z_mean.append(np.mean(fig[2,:]))
  #print di verifica andamento
  '''print("num fig: ", i)
  print("MEAN: x", x_mean[i], "y", y_mean[i], "z", z_mean[i])
  print("MIN: x", np.max(fig[0,:]), "y", np.max(fig[1,:]), "z", np.max(fig[2,:]))
  print("MAX: x", np.min(fig[0,:]), "y", np.min(fig[1,:]), "z", np.min(fig[2,:]))'''


  #verifica normalizzazione
  if max < 1.01 and min > -1.01:
    pass
  else:
    count +=1
    print("count", count)
    print("max", max)
    print("min", min)
#print(pointcloud)
print(count)




'''if i>=7208:
    print(fig.shape)
  fig = pointcloud[i, :, :]
  max = np.max(fig)
  min = np.min(fig)
  fig = fig/max
  print(min)
  print(max)
print(fig.shape)
print(fig)'''

In [ ]:
#istogrammi di controllo medie dati originali

print("sum mean x", sum(x_mean))
print("sum mean y", sum(y_mean))
print("sum mean z", sum(z_mean))


plt.plot(range(len(x_mean)), x_mean, label='x_mean',alpha=0.2)
plt.plot(range(len(y_mean)), y_mean, label='y_mean',alpha=0.2)
plt.plot(range(len(z_mean)), z_mean, label='z_mean', )
plt.legend()
plt.show()
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.hist(x_mean, label='x_mean')
plt.subplot(1,3,2)
plt.hist(y_mean, label='y_mean')
plt.subplot(1,3,3)
plt.hist(z_mean, label='z_mean')
plt.legend()
plt.show()

In [ ]:
#plot di una figura 3d (non normalizzata)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Genera dati per una sfera unitaria
phi = np.linspace(0, np.pi, 100)
theta = np.linspace(0, 2 * np.pi, 100)
phi, theta = np.meshgrid(phi, theta)
x = np.sin(phi) * np.cos(theta)
y = np.sin(phi) * np.sin(theta)
z = np.cos(phi)

print(type(fig[0,:]))
# Crea una figura tridimensionale
figure = plt.figure()
ax = figure.add_subplot(111, projection='3d')
print(type(x))
# Plotta la sfera unitaria
ax.scatter(fig[0,:], fig[1,:], fig[2,:], c = 'r')
ax.plot_surface(x, y, z, color='b', alpha=0.2)



# Imposta il rapporto di scala in modo che i dati siano visualizzati in modo corretto
ax.set_aspect('auto')
ax.axis('equal')
# Mostra il grafico
plt.show()

#### Normalizzazione (vera e propria)

In [ ]:
#centramento e poi riscalamento
pc_norm = np.zeros(pointcloud.shape)
print(pointcloud.shape)
print(pc_norm.shape)

for i in range(pointcloud.shape[0]):
    fig = pointcloud[i, :, :]
    pc_norm[i, 0, :] = fig[0]-x_mean[i]
    pc_norm[i, 1, :] = fig[1]-y_mean[i]
    pc_norm[i, 2, :] = fig[2]-z_mean[i]

    #riscalamento figura
    max_dist = np.max(np.linalg.norm(pc_norm[i,:,:], axis=0))
    pc_norm[i,:,:] /= max_dist

In [ ]:
#nuova verifica dopo la normalizzazione dei dati
x_mean = []
y_mean = []
z_mean = []

for i in range(pc_norm.shape[0]):
  fig = pc_norm[i, :, :]
  #print("fig.shape = ", fig.shape)

  max = np.max(fig)
  min = np.min(fig)

  #verifica centramento
  x_mean.append(np.mean(fig[0,:]))
  y_mean.append(np.mean(fig[1,:]))
  z_mean.append(np.mean(fig[2,:]))
  '''print("num fig: ", i)
  print("MEAN: x", x_mean[i], "y", y_mean[i], "z", z_mean[i])
  print("MIN: x", np.max(fig[0,:]), "y", np.max(fig[1,:]), "z", np.max(fig[2,:]))
  print("MAX: x", np.min(fig[0,:]), "y", np.min(fig[1,:]), "z", np.min(fig[2,:]))'''


  #verifica normalizzazione
  if max < 1.01 and min > -1.01:
    pass
  else:
    count +=1
    print("count", count)
    print("max", max)
    print("min", min)

#print(pointcloud)
print(count)

In [ ]:
#istogrammi di verifica
print("sum mean x", sum(x_mean))
print("sum mean y", sum(y_mean))
print("sum mean z", sum(z_mean))


plt.plot(range(len(x_mean)), x_mean, label='x_mean',alpha=0.2)
plt.plot(range(len(y_mean)), y_mean, label='y_mean',alpha=0.2)
plt.plot(range(len(z_mean)), z_mean, label='z_mean', )
plt.legend()
plt.show()
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.hist(x_mean, label='x_mean')
plt.subplot(1,3,2)
plt.hist(y_mean, label='y_mean')
plt.subplot(1,3,3)
plt.hist(z_mean, label='z_mean')
plt.legend()
plt.show()

Plot 3d figura normalizzata

In [ ]:
# Genera dati per una sfera unitaria
phi = np.linspace(0, np.pi, 100)
theta = np.linspace(0, 2 * np.pi, 100)
phi, theta = np.meshgrid(phi, theta)
x = np.sin(phi) * np.cos(theta)
y = np.sin(phi) * np.sin(theta)
z = np.cos(phi)

print(type(fig[0,:]))
# Crea una figura tridimensionale
figure = plt.figure()
ax = figure.add_subplot(111, projection='3d')
print(type(x))
# Plotta la sfera unitaria
ax.scatter(fig[0,:], fig[1,:], fig[2,:], color = 'r', marker = '.')
ax.plot_surface(x, y, z, color='b', alpha=0.2)



# Imposta il rapporto di scala in modo che i dati siano visualizzati in modo corretto
ax.set_aspect('auto')
ax.axis('equal')
# Mostra il grafico
plt.show()

### SPLIT pointcloud in train, validation e test
con TensorData e DataLoader e creazione dei batch

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.functional import normalize
import torch


train_pointcloud, t_pointcloud, train_label, t_label = train_test_split( pc_norm, labels, train_size=0.6, shuffle = True, random_state=1234)
test_pointcloud, validation_pointcloud, test_label, validation_label = train_test_split( t_pointcloud, t_label, train_size=0.5, shuffle = True, random_state=5678)

print(train_pointcloud.shape, train_label.shape)
print(test_pointcloud.shape, test_label.shape)
print(validation_pointcloud.shape, validation_label.shape)

print(type(train_pointcloud[0,0,0]))

train_pt=torch.Tensor(train_pointcloud).float()
test_pt=torch.Tensor(test_pointcloud).float()
validation_pt=torch.Tensor(validation_pointcloud).float()
train_label_pt=torch.Tensor(train_label).int()
test_label_pt=torch.Tensor(test_label).int()
validation_label_pt=torch.Tensor(validation_label).int()

dataset_train = TensorDataset(train_pt,train_label_pt)
dataset_validation = TensorDataset(validation_pt,validation_label_pt)
dataset_test = TensorDataset(test_pt,test_label_pt)

dataloader_train = DataLoader(dataset_train, batch_size = batchsize, shuffle = True, drop_last = True)
dataloader_validation = DataLoader(dataset_validation, batch_size = batchsize, shuffle = True, drop_last = True)
dataloader_test = DataLoader(dataset_test, batch_size = batchsize, shuffle = True, drop_last = True)

print(train_pt.shape, train_label_pt.shape)
print(type(train_pt))
print(type(dataset_train))


In [ ]:
print(dataloader_train)

### Trasferimento su cuda dei dataset

In [ ]:
if torch.cuda.is_available():
  print('number of devices: ', torch.cuda.device_count())
  print(torch.cuda.get_device_name(0))
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Computation device: {device}\n")

print(type(train_pt))
print(train_pt.shape)
print('X_train_pt is in GPU:', train_pt.is_cuda)
train_pt = train_pt.to(device)
print('X_train_pt is in GPU:', train_pt.is_cuda)
print(type(train_pt))
print(train_pt.shape)

## MODELLO

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F


class STN3d(nn.Module):
    def __init__(self):
        super(STN3d, self).__init__()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 9)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)


    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.array([1,0,0,0,1,0,0,0,1]).astype(np.float32))).view(1,9).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, 3, 3)
        return x


class STNkd(nn.Module):
    def __init__(self, k=64):
        super(STNkd, self).__init__()
        self.conv1 = torch.nn.Conv1d(k, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k*k)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

        self.k = k

    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.eye(self.k).flatten().astype(np.float32))).view(1,self.k*self.k).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

class PointNetfeat(nn.Module):
    def __init__(self, global_feat = True, feature_transform = False):
        super(PointNetfeat, self).__init__()
        self.stn = STN3d()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.global_feat = global_feat
        self.feature_transform = feature_transform
        if self.feature_transform:
            self.fstn = STNkd(k=64)

    def forward(self, x):
        n_pts = x.size()[2]
        trans = self.stn(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans)
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))

        if self.feature_transform:
            trans_feat = self.fstn(x)
            x = x.transpose(2,1)
            x = torch.bmm(x, trans_feat)
            x = x.transpose(2,1)
        else:
            trans_feat = None

        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        if self.global_feat:
            return x, trans, trans_feat
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, n_pts)
            return torch.cat([x, pointfeat], 1), trans, trans_feat

class PointNetCls(nn.Module):
    def __init__(self, k=2, feature_transform=False):
        super(PointNetCls, self).__init__()
        self.feature_transform = feature_transform
        self.feat = PointNetfeat(global_feat=True, feature_transform=feature_transform)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k)
        self.dropout = nn.Dropout(p=drop_rate)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()

    def forward(self, x):
        x, trans, trans_feat = self.feat(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.dropout(self.fc2(x))))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1), trans, trans_feat

def feature_transform_regularizer(trans):
    d = trans.size()[1]
    batchsize = trans.size()[0]
    I = torch.eye(d)[None, :, :]
    if trans.is_cuda:
        I = I.cuda()
    loss = torch.mean(torch.norm(torch.bmm(trans, trans.transpose(2,1)) - I, dim=(1,2)))
    return loss

## Training Modello

In [ ]:
!pip install torchmetrics
!pip install torcheval

training modello

In [ ]:
from __future__ import print_function
import os
import time
import random
import torch
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
#from pointnet.model import PointNetCls, feature_transform_regularizer
import torch.nn.functional as F
from tqdm import tqdm
import torchmetrics
from torcheval.metrics.functional import multiclass_f1_score

metric_func = multiclass_f1_score


manualSeed = 1234  # fix seed
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)


print(len(dataset_train), len(dataset_validation))
num_classes = 2
print('classes', num_classes)

try:
    os.makedirs(outf)
except OSError:
    pass

classifier = PointNetCls(k=num_classes, feature_transform=feature_transform)

optimizer = optim.Adam(classifier.parameters(), lr = optimizer_learning_rate, betas = optimizer_beta)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = scheduler_step_size, gamma=  scheduler_gamma)
classifier.cuda()

loss_train = []
acc_train = []
loss_val = []
acc_val = []
metric_train = []
metric_val = []
time_perepoch = []


num_batch = len(dataset_train) / batchsize
print(num_batch)
for epoch in range(nepoch):
    t0 = time.time()
    scheduler.step()

    train_loss = 0
    valid_loss = 0
    train_counter = 0
    valid_counter = 0
    train_classification = 0
    valid_classification = 0
    valid_metric = 0
    train_metric = 0

    #train
    for i, data in enumerate(dataloader_train, 0):
        train_counter += 1
        points, target = data
        #points = points.transpose(2, 1) #lascia la prima dimensione invariata, traspone solo le altre altre
        points, target = points.cuda(), target.cuda()
        optimizer.zero_grad()
        classifier = classifier.train()
        pred, trans, trans_feat = classifier(points)
        target = target.type(torch.long)
        loss = F.nll_loss(pred, target) #per problemi multiclasse
        #pred_cls = pred[:,1].float()
        #loss = loss_BCE(pred, target.float()) #per probl a 2 classi
        if feature_transform: #ancora non si è capito perchè? runnare
            loss += feature_transform_regularizer(trans_feat) * 0.001
        train_loss += loss.item()

        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.data).cpu().sum()
        #print('[%d: %d/%d] train loss: %f accuracy: %f' % (epoch, i, num_batch, loss.item(), correct.item() / float(batchsize)))
        train_classification += correct.item()
        #metric
        metric = metric_func(pred,target)
        train_metric += metric.item()

        #backpropagation
        loss.backward()
        optimizer.step()
        #optimizer.zero_grad() #sta gia sopra!?

    #validation
    with torch.no_grad():
        for j, data in enumerate(dataloader_validation, 0):
            valid_counter += 1
            points, target = data
            points, target = points.cuda(), target.cuda()
            classifier = classifier.eval()
            pred, _, _ = classifier(points)
            target = target.type(torch.long)
            loss = F.nll_loss(pred, target)
            #pred_cls = pred[:,1].float() #prendiamo i valori di prob max e li trasformiamo in float(lo [0] è per prendere solo i valori e non gli indici)
            #loss = loss_BCE(pred_cls, target.float())
            valid_loss += loss.item()
            pred_choice = pred.data.max(1)[1]
            correct = pred_choice.eq(target.data).cpu().sum()
            valid_classification += correct.item()
            #print('[%d: %d/%d] %s loss: %f accuracy: %f' % (epoch, i, num_batch, 'test', loss.item(), correct.item()/float(batchsize)))
            #metric
            metric = metric_func(pred,target)
            valid_metric += metric.item()
    elapsed_time = time.time() - t0
    time_perepoch.append(elapsed_time)

    train_loss /= train_counter
    loss_train.append(train_loss)
    valid_loss /= valid_counter
    loss_val.append(valid_loss)
    train_classification /= (i*batchsize)
    acc_train.append(train_classification)
    valid_classification /= (j*batchsize)
    acc_val.append(valid_classification)
    train_metric /= train_counter
    metric_train.append(train_metric)
    valid_metric /= valid_counter
    metric_val.append(valid_metric)
    print("epoch: %d, time: %.4f, train_loss: %.4f, valid_loss: %.4f, train_metric: %.4f, valid_metric: %.4f, train_accuracy: %.4f, valid_accuracy: %.4f" % (epoch, elapsed_time, train_loss, valid_loss, train_metric, valid_metric, train_classification, valid_classification))


    torch.save(classifier.state_dict(), '%s/cls_model_%d.pth' % (outf, epoch))


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,5))
plt.title("Training and Validation curves")
plt.suptitle("np=%d nb=%d ne=%d dr=%.2f dataset Sbilanciato\n optimizer: lr=%.4f b=(%.2f,%.4f) scheduler: ss=%d g=%.2f" % (num_points, batchsize, nepoch, drop_rate, optimizer_learning_rate, optimizer_beta[0], optimizer_beta[1], scheduler_step_size, scheduler_gamma))

plt.subplot(1,3,1)
plt.grid()
plt.title("LOSS")
plt.plot(range(len(loss_train)),loss_train, 'r', label='train')
plt.plot(range(len(loss_val)),loss_val,'b',label='vali')
plt.legend()
plt.ylabel("Loss")

plt.subplot(1,3,2)
plt.grid()
plt.title("METRIC")
plt.plot(range(len(metric_train)),metric_train, 'r',label='train')
plt.plot(range(len(metric_val)),metric_val,'b',label='vali')
plt.ylabel("Metric")

plt.subplot(1,3,3)
plt.grid()
plt.title("ACCURACY")
plt.plot(range(len(acc_train)),acc_train, 'r',label='train')
plt.plot(range(len(acc_val)),acc_val,'b',label='vali')
plt.ylabel("Accuracy")

plt.grid()
plt.legend()
plt.tight_layout()
#plt.show()
nome_png = nome_file + ".png"
plt.savefig(nome_png)
#files.download(nome_png)

## Test modello

In [ ]:
total_correct = 0
total_testset = 0
for i,data in tqdm(enumerate(dataloader_test, 0)):
    points, target = data
    points, target = points.cuda(), target.cuda()
    classifier = classifier.eval()
    pred, _, _ = classifier(points)
    pred_choice = pred.data.max(1)[1]
    correct = pred_choice.eq(target).cpu().sum()
    total_correct += correct.item()
    total_testset += points.size()[0]

test_accuracy = total_correct / float(total_testset)
print("final accuracy {}".format(total_correct / float(total_testset)))

## Output

In [ ]:
import pandas as pd
nome_csv = nome_file + ".csv"
frame = np.array([ range(nepoch), loss_train, acc_train, loss_val, acc_val, metric_train, metric_val, time_perepoch])
frame = np.transpose(frame)
df = pd.DataFrame(frame, columns = [ "nepoch", "loss_train", "acc_train", "loss_val", "acc_val", "metric_train", "metric_val", "time_perepoch"])
display(df)
df.to_csv(nome_csv)

In [ ]:
nome_txt = nome_file + ".txt"
print(nome_txt)
with open(nome_txt, 'w') as f:
    f.write('IPERPARAMETRI\n- dati sbilanciati')
    f.write("\n- num_points = " + str(num_points))
    f.write('\n- batchsize = '+ str(batchsize))
    f.write('\n- feature_transform = '+ str( feature_transform))
    f.write('\n- optimizer_learning_rate = '+ str(optimizer_learning_rate))
    f.write('\n- optimizer_beta = '+ str(optimizer_beta))
    f.write('\n- scheduler_step_size = ' + str(scheduler_step_size))
    f.write('\n- scheduler_gamma = '+ str(scheduler_gamma))
    f.write('\n- drop_rate = '+ str(drop_rate))
    f.write('\n- workers = '+ str(workers))
    f.write('\n- nepoch = '+ str(nepoch))
    f.write('\n------------------------------------------------------')
    f.write('\nTEST FINAL ACCURACY')
    f.write('\ntest_accuracy = %.4f'+ str(test_accuracy))
    f.write('\n------------------------------------------------------')
f.close()

In [ ]:
from google.colab import files
files.download(nome_png)
files.download(nome_csv)
files.download(nome_txt)